# A Guide for Your First Micromagnetic Simulations

**The purpose of this notebook is to give anyone new to the subject a quick start to running their first micromagnetic simulations.** In addition, this notebook provides 
you with a couple of useful tools. For example, mounting a remote file system allows you to access files on a remote server directly via a graphical user interface on your local machine.

## 1. Connecting to the remote server

To run simulations on the VSC (or GTO), you first have to establish a connection to its servers. Here's how to do it:

1. First, you have to make sure that you can use the virtual private network, or **VPN**, of the university of Vienna. Follow the step-by-step instructions on this page to get started: https://zid.univie.ac.at/vpn/#c41775
2. In addition, you need a **personal VSC account with a password**. Ask Dieter or Claas to create such an account for you. You can change the password later on.  
3. Once you have taken care of this, make sure to connect to the university VPN using BIG-IP Edge Client to start your session. A two-factor authentication is required, so keep your phone readily available. 
4. Next open a terminal and type `ssh user_name@vsc5.vsc.ac.at` for establishing a connection to VSC or `ssh user_name@131.130.25.48` for GTO. Of course, you have to substitute your actual user name instead of typing `user_name`.
5. You are then prompted for your personal password as well as a code you receive via SMS. If successful, you are logged in and ready to get your work started! 
6. Move to the folder assigned to you (e.g. called adam) in the terminal and create a folder using e.g. `mkdir data`. **This is the place to keep your files. Make sure to work exclusively in your personal folder**.

## 2. Managing your files

At some point, you might figure that it is more convenient to manage all the data you have on your VSC directly from your local machine, instead of having to do all of this from within your terminal. You can achieve this by using FUSE (file system in user space). FUSE allows you to mount a remote file system like yours on the VSC. And it's as easy as this:

1. Install the software:
    - Mac: `https://osxfuse.github.io`
    - Windows: `https://github.com/winfsp/winfsp`
    - Linux: `https://wiki.ubuntuusers.de/FUSE/`
2. Next, make sure you have created an empty folder, in which the VSC data should be mounted. For example, create a folder `VSC_data` on your desktop. 
3. To acutally mount, type `sshfs -o follow_symlinks user_name@vsc5.vsc.ac.at:/home/fs71388/user_name VSC_data` in the parent folder of `VSC_data`. Note that there is no blank space between the ssh connection and the path, but there is one between the path and the folder name. 
4. If this went fine, you should be prompted to type the password you were assigned for the VSC or GTO. 
5. You should now be able to see all the data you have, e.g. on the VSC, in the local `VSC_data` folder on your desktop. Now you can access all scripts, plots or data you have produced in your simulation locally within your `VSC_data` folder.

Now, there are a couple of things that can go wrong. For example, if you have a typo in the above command, you might get an error message saying something like this: `mount_macfuse: mount point /path_to_your_VSC_folder/VSC_data is itself on a macFUSE volume`. To fix it, you first have to unmount by saying `umount path_to_your_VSC_folder/VSC_data` and go back to step 2. of the above recipe. Another problem you might get is that the terminal itself might not react anymore and you have to restart the computer (at least this was the case on MacOS). So make sure not to make a mistake here.

In fact, there is also a simpler approach to move files from your computer to the VSC (or GTO). Instead of mounting the remote file system, you can just copy around files 'manually' by proceeding as follows:

1. Open a terminal on the local machine where the file is located or should be moved to.

2. For moving a file from the local machine to the external server: `scp path_on_local_machine/filename.py user_name@vsc5.vsc.ac.at: path_on_external_server/filename.py`

3. For moving a file from the external server to the local machine: `scp user_name@vsc5.vsc.ac.at: path_on_external_server/filename.py path_on_local_machine/filename.py`

## 3. Managing jobs on a high-performance computer

A lot of people run their simulations on VSC or GTO. So to submit and manage jobs (i.e. computational tasks) in an organized and efficient way, one can use so-called Slurm scripts. In this way, many users can run their simulations at the same time and without conflict. In this section you learn how use Slurm scripts to submit a job on the VSC.

Let's have a closer look at a prototypical Slurm script that is used for running simulations on the VSC. Asterisked explanations are inspired by ChatGPT output.

- First, you specify the script interepreter by typing `#!/bin/bash`. In this case, `bash`.
- Next, the file contains so-called Slurm directives. Lines starting with #SBATCH are Slurm directives, providing information to the Slurm scheduler:

    -   `#SBATCH -J your_sim_name`
    
        [Substitute the name of your simulation for your_sim_name]
    -   `#SBATCH -n 1`

        [These are the number of cores you request for a simulation, in this case it's one core.]
    -   `#SBATCH --time = "24:00:00"`

        [Type in how long the simulation runs (24h here). If you know how long it runs in advance, e.g. if you
        know it just takes ten minutes, you can type time = "00:10:00". If possible, the simulation will then be prioritized and 
        run before other simulations in the queue, which take longer than your own simulation. Note that your job will be cancelled if your simulation takes
        longer than specified in `time`.]
    -   `#SBATCH --partition = zen3_0512_a100x2`

        [This lines specifies the partition. Here, it is a specific graphic card, on which the simulation is run. a100x2 denotes the large GPU, which
        should only be used for largeer simulations. For smaller ones use zen2_0256_a40x2.]
    
    -   `#SBATCH --qos = zen3_0512_a100x2`
        
        [When you specify --qos = normal in a Slurm script, you are indicating that you want to assign the "normal" Quality of Service to your job. Different clusters may have different QoS levels, each with its own set of characteristics or policies. The "normal" QoS is typically a standard or default level of service provided by the cluster. Again, for smaller simulations use zen2_0256_a40x2.]*

    -   `#SBATCH --mail-type = ALL`
        `#SBATCH --mail-user = <user_name@univie.ac.at>``

        [By typing these lines you activate email notification when the job starts, ends or fails. Moreover, you specify the recipient of the notifiaction mail. In fact, this
        is useful to track the progress of the simulations you've started.]*
    
    -   `#SBATCH --requeue`

        [The --requeue option in a Slurm script is used to request that if a job is terminated due to a node failure or other reasons, Slurm should attempt to requeue the job and run it again. Essentially, it allows the job to be resubmitted automatically in the case of certain failures.]

    -   `#SBATCH --gres = gpu:1 `

        [When you include --gres = gpu:1 in your Slurm script, you are telling the Slurm scheduler that your job needs access to a GPU for its execution. The scheduler will then try to allocate a node with an available GPU to your job when resources become available. If your application or task requires more GPUs, you can adjust the number accordingly (e.g., gpu:2 for two GPUs).]

    -   `#SBATCH --array = 0-9`

        [If you have a set of ten parameters you want to do the simulation with, type array = 0-9; for a set of five parameters type array = 0-4 etc.]

    -   `#SBATCH -o SlurmOutput`

        [Specifies the name of the output file.]

- So to set up a Slurm script create a file named for example `jobs_gpu.slrm` and adjust the parameters above to your needs. For example,

    ```
    #SBATCH -n 1
    #SBATCH --time = "00:10:00"
    #SBATCH --partition = zen2_0256_a40x2
    #SBATCH --qos = zen2_0256_a40x2
    #SBATCH --mail-type=ALL
    #SBATCH --requeue
    #SBATCH --gres = gpu:1 
    #SBATCH --array = 0-5
    #SBATCH -o SlurmOutput
    ```
-  Next insert the following lines of code. This is the part you will probably not change very often. Adapt the scriptname `run` and `output_folder` to your needs.

    ```

    # Number of tasks per node

    TASKS_PER_NODE = 1

    # Initialize two arrays 

    RANKs = ()      
    PIDs = ()

    # This line activates a virtual environment located at the specified path
    
    source /home/fs71388/amil4/pi.magnum_vsc4/data/public_magnumnp_V114/myenv/bin/activate

    # Define the scriptname as 'run' and a folder name output_folder to specify where the simulation results are saved
    scriptname = run 
    logs=logs_${scriptname}
    mkdir -p $logs
    #-o output_folder

    # Iterates over all values for RANK 

    for i in `seq $TASKS_PER_NODE`
    do
    RANKs[$i]=$(printf "%06d" $(($SLURM_ARRAY_TASK_ID * $TASKS_PER_NODE + $i - 1)))
    python3 ${scriptname}.py --rank=${RANKs[$i]} --output output_folder > ${logs}/run_${RANKs[$i]}.out 2> ${logs}/run_${RANKs[$i]}.err &
    PIDs[$i]=$!
    echo "Starting Rank ${RANKs[$i]} (PID: ${PIDs[$i]})"
    done

    # Wait for background jobs

    for i in `seq $TASKS_PER_NODE`
    do
    wait ${PIDs[$i]}
    echo "Rank ${RANKs[$i]} (PID: ${PIDs[$i]}): finished with return value $?"
    done
    ```

- Now that your Slurm script is ready, you can perform more complex simulations on the VSC. One advantage is that you can let your simulation be run automatically for a couple of different sets of parameters. To sketch how this works, consider this:

    1. Use the argparse module in your python file to define a parameter called `rank`: 
    
         ```
        parser = argparse.ArgumentParser(description = 'Run simple simulation.')
        parser.add_argument('-r', '--rank', default = 0, type = int)
        parser.add_argument('-o', '--output', default = "output_folder", type = str)
        args = parser.parse_args()
        outdir = args.output
        ```

    2. For each parameter value of `rank`, a specific set of parameters for the simulation can then be associated. For example, if `rank` is 0 the simulation should run for the parameters $A = 1$, $B = 2$ and $C = 3$. If rank is 1, it should run the simulation for the values $A = 4, B = 5, C = 6$ and so on. The parameters could be stored in a dictionary within the main file, but they could also be read in from a vsc or excel file.
    3. Once this is fixed in the script, move to the slrm script and adjust its parameters if needed (for example, the running time).

- Finally, if everything is set up, you can submit the job using: `sbatch jobs_gpu.slrm` (Replace jobs by the name of the slrm script.) If necessary, the job can be cancelled using `scancel JOBID`, where JOBID is a number associated to the submitted job. If you want to look the number up, use: `qs`. To get more detailed information about who is running which job on which GPU you can type `qs_gpu_all`.

    - Note that the latter commands first need to be defined in the .bashrc file. To do so, go to your home folder on the VSC cluster and type  `vim .bashrc`. Within this file you then need to adjust the lines `alias qs_gpu_all = "squeue | grep 0x2" ` and `alias qs = "squeue -uuser_name"`. Of course you could define them differently, if you wanted.

- Please use `source /home/fs71388/amil4/pi.magnum_vsc4/data/public_magnumnp_V114/myenv/bin/activate` in your Slurm script. Amil set this up and remarked: 

    "We currently don't have a unified solution for simulating with magnum.np
    on VSC5. The solution up to now was that everyone who simulates has
    their own local installation of magnum.np. However this is not feasable,
    since we have very little space on /home of VSC5. It is also a bit
    unintuitive to do this setup alone since you need a specific version of
    torch in order for magnum.np to calculate on the GPUs. Therefore I tried
    to create a virtual environment, to which everyone has access to, where
    all the correct packages are already installed.
    The idea is that you only need to log on, load the environment, and
    everything should be working. Attached is a SLURM script in which the
    environment is being loaded, before the simulations are started.
    If you have questions or feel that this could be improved in a way, feel
    free to contact me!
    
    PS: If you are new to simulating on the VSC5 please keep in mind
    1.) to store all simulation data in /gpfs/data/fs71388/user since as
    already mentioned /home is quite small
    2.) to keep track of how many hours you are simulating, your statistics
    can be seen here https://service.vsc.ac.at/clusteruser/stats/6"


## 4. Micromagnetic Simulations

Check out the Gitlab page of Magnumnp to download some demos and run your first micromagnetic simulations on VSC: `https://gitlab.com/magnum.np/magnum.np/-/tree/main/demos?ref_type=heads`. From the physical perspective, the following paper is a good place to get started: `https://scholar.google.at/citations?view_op=view_citation&hl=de&user=di9SqgIAAAAJ&citation_for_view=di9SqgIAAAAJ:O3NaXMp0MMsC`.

## Appendix A: Navigating the terminal like a pro

This is a list of some terminal commands you might find useful. The ones you'll most frequently use are, of course, `cd` to change the directory and `ls` to list all files and folders in the current directory. Many IDEs like Visual Studio Code provide an emulator of your local terminal, which you can use in the same way. The following list was partially generated by ChatGPT and provides Linux/Mac terminal commands. So if you're a Windows user, you'll have to adapt the list because the commands are different.

1. **Use Tab Completion (very important)**
   - Press `Tab` to auto-complete commands, file names, and directories. It saves time and minimizes typing errors.

2. **Creating Files and Folders (very important)**
   - To create a folder, type: `mkdir your_folder_name`
   - To create a new file type: `touch your_file_name.txt`, `touch run.py` etc.

   3. **Quick Directory Change: (very important) **
   - Use `cd` to change directories. Typing `cd -` takes you to the previous directory, and `cd ~` goes to your home directory.
   - To navigate very quickly to your home directory type from anywhere in the terminal `cd ~``
   - To navigate to a parent folder type: `cd ../` 

3. **Copy files and folders (very important)**
   - To copy a file type: `cp your_file_name.txt your_file_name_copy.txt``
   - To copy a file from a specific folder to another one type: ` cp path_to_file_in_specific_folder/file_name.txt path_to_another_folder/file_name.txt`
   - You can also copy a folder like this: `cp old_folder_name new_folder_name`

   
2. **History Navigation:**
   - Press the up and down arrow keys to cycle through your command history. Combine with `Ctrl + R` to search backward in history.
   - This command allows to browse through the history and to search for the keyword "yourkeyword" in the command line history:
   `history | grep -i yourkeyword`

4. **Shortcut for Home and End:**
   - Press `Ctrl + A` to move to the beginning of the line and `Ctrl + E` to move to the end. Handy for quick edits.
   
5. **Deleting Content of a Line:**
   - Press `Ctrl + U` to delete from the cursor to the beginning of the line.
   - Press `Ctrl + K` to delete from the cursor to the end of the line.

6. **Clearing the Terminal:**
   - Press `Ctrl + L` to clear the terminal screen. This doesn't delete the command history but provides a clean slate.

7. **Wildcard Characters:**
   - Employ `*` for wildcard matching. For instance, `ls *.txt` lists all text files in the current directory.

8. **Recursive Commands:**
   - Many commands support the `-r` flag for recursive operations. For instance, to recursively list all files in a directory, you can use:

     ```
     ls -R
     ```

   - To recursively remove a directory and its contents, use:

     ```
     rm -r directory_name
     ```

9. **Pipeline and Redirection:**
   - Use `|` to pipe the output of one command as the input to another. For example, `ls | grep keyword` filters directory content.
   - Use `>` to redirect output to a file. For example, `ls > file_list.txt` saves the list of files to a file.

10. **Ctrl + C and Ctrl + D:**
   - Press `Ctrl + C` to interrupt a command and `Ctrl + D` to signal the end of input or exit a shell session.

11. **Multiple Windows/Tabs:**
   - If using a terminal emulator like `tmux` or `screen`, explore multiple windows or tabs for simultaneous tasks. 

## Appendix B: Editing your code 

Some people like to use an editor called **vim** to edit their code. For example, if you want to open a file `run.py` on your VSC repository and make some changes to it, you can type `vim run.py`. Here is a list for the most important commands in `vim` that was generated by ChatGPT. Alternatively, you can use an IDE like **Visual Studio Code**, with which you can also access the VSC server and that allows for a more intuitive navigation in your files. Moreover, you can use a bunch of built-in tools like that make your live as a programmer easier. This is probably a matter of taste…

### Basic Navigation
1. **Move Cursor:**
    - `h`: Move left
    - `j`: Move down
    - `k`: Move up
    - `l`: Move right

2. **Navigate by Words:**
    - `w`: Move to the beginning of the next word
    - `b`: Move to the beginning of the previous word
    - `e`: Move to the end of the current word

3. **Navigate by Lines:**
    - `0`: Move to the beginning of the line
    - `$`: Move to the end of the line
    - `gg`: Move to the beginning of the file
    - `G`: Move to the end of the file
    - `:<line_number>`: Move to a specific line number

### Editing

4. **Insert Mode:**
    - `i`: Insert before the cursor
    - `I`: Insert at the beginning of the line
    - `a`: Insert after the cursor
    - `A`: Insert at the end of the line
    - `o`: Open a new line below the current line
    - `O`: Open a new line above the current line

5. **Editing Text:**
    - `x`: Delete the character under the cursor
    - `dd`: Delete the current line
    - `yy`: Yank (copy) the current line
    - :%y: Copy all lines in a file 
    - `p`: Paste the yanked or deleted text after the cursor
    - `u`: Undo
    - `Ctrl + r`: Redo

### Searching and Replacing

6. **Search:**
    - `/pattern`: Search forward for a pattern
    - `?pattern`: Search backward for a pattern
    - `n`: Move to the next occurrence
    - `N`: Move to the previous occurrence

7. **Replace:**
    - `:s/old/new`: Replace the first occurrence of "old" with "new" on the current line
    - `:s/old/new/g`: Replace all occurrences of "old" with "new" on the current line
    - `:%s/old/new/g`: Replace all occurrences in the entire file

### Saving and Exiting

8. **Save and Quit:**
    - `:w`: Save changes
    - `:wq` or `ZZ`: Save changes and quit

9. **Quit without Saving:**
    - `:q`: Quit (only if no changes were made)
    - `:q!`: Quit without saving changes

### Miscellaneous

10. **Copy/Paste (Visual Mode):**
    - `v`: Enter visual mode
    - `V`: Enter visual line mode
    - `Ctrl + v`: Enter visual block mode

11. **Indentation:**
    - `>>`: Indent the current line
    - `<<`: Unindent the current line

12. **Exiting and Entering Insert Mode:**
    - `Esc`: Exit insert mode or visual mode and return to normal mode
    - `i`: Enter insert mode before the cursor
    - `I`: Enter insert mode at the beginning of the line
    - `a`: Enter insert mode after the cursor
    - `A`: Enter insert mode at the end of the line

13. **Exiting Vim:**
    - `:q`: Quit (if no changes were made)
    - `:q!`: Quit without saving changes
    - `:wq` or `ZZ`: Save changes and quit


## Appendix C: Screen

**Screen** allows you to continue simulations on an external server even if for some reason the connection is lost. When you are using slrm scripts on the VSC, this tool 
is not really needed, but it can be useful for simulations running on GTO since the simulation stops when you close the terminal (for example when you go home in the evening). 
So here is a crash course on this tool.


1. Type `screen` and hit enter; now you are in a screen. 
2. Start a script, e.g. by typing `python3 run.py`.
3. Detach from the screen associated by hitting `ctrl+a` and then `d`
4. To list all screen sessions type screen `-ls`
5. To reattach to a session type screen `-r sessionname`
6. To kill a screen session type screen `-X -S sessionname quit`
7. To rename a screen session hit `ctrl+a` within a screen window, then type a colon `:` to open up a command line; afterwards type `sessionname new_session_name`. 